In [1]:
# ============================================================
# Cell 0 — CONFIG, FOLDERS, LOGGING
# ------------------------------------------------------------
# - Train set:   images/problem/{task1, task2}
# - Public test: images/test  (PNG trộn cả 2 task)
# - Private:     ./private_test (ở root; có thể bị khoá)
# - Sau TRAIN: đóng băng (freeze) tham số -> INFER không học nữa.
# ============================================================

import os, sys, glob, time, math, json, logging, zipfile, heapq, itertools, random
from collections import defaultdict
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm

def ensure_dir(path: str) -> bool:
    """Tạo thư mục nếu chưa có; trả False nếu tạo thất bại (đã log ra stdout)."""
    try:
        os.makedirs(path, exist_ok=True)
        return True
    except Exception as e:
        print(f"[ERROR] Cannot create dir {path}: {e}", flush=True)
        return False

def require_dir(path: str, create: bool = False) -> str:
    """
    Đảm bảo thư mục tồn tại; nếu create=True thì cố gắng tạo, không được thì ném lỗi.
    Dùng cho INPUT (create=False) & OUTPUT (create=True).
    """
    if os.path.isdir(path):
        return path
    if create and ensure_dir(path):
        return path
    raise FileNotFoundError(f"Required directory not found: {path}")

# ---- Base path (nếu notebook nằm trong code/ thì lùi 1 cấp) ----
CWD = os.getcwd()
BASE_DIR = os.path.abspath(os.path.join(CWD, "..")) if os.path.basename(CWD) == "code" else os.path.abspath(CWD)

# ---- TRAIN SET (images/problem) ----
IMAGES_DIR      = os.path.join(BASE_DIR, "images")
TRAIN_TASK1_DIR = require_dir(os.path.join(IMAGES_DIR, "problem", "task1"), create=True)   # có thể rỗng
TRAIN_TASK2_DIR = require_dir(os.path.join(IMAGES_DIR, "problem", "task2"), create=True)   # có thể rỗng

# ---- TEST SET ----
TEST_PUBLIC_DIR  = require_dir(os.path.join(IMAGES_DIR, "test"), create=True)              # public_test
PRIVATE_TEST_DIR = os.path.join(BASE_DIR, "private_test")                                  # có thể chưa mở khoá

# ---- OUTPUT & SOLVED ----
SLOVED_TASK1   = require_dir(os.path.join(IMAGES_DIR, "sloved", "task1"), create=True)     # giữ nguyên "sloved"
SLOVED_TASK2   = require_dir(os.path.join(IMAGES_DIR, "sloved", "task2"), create=True)
SLOVED_PUBLIC  = require_dir(os.path.join(IMAGES_DIR, "sloved", "public_test"), create=True)
SLOVED_PRIVATE = require_dir(os.path.join(IMAGES_DIR, "sloved", "private_test"), create=True)
OUTPUT_DIR     = require_dir(os.path.join(BASE_DIR, "output"), create=True)

# ---- ĐẶC TẢ đề bài ----
GRID_R, GRID_C = 3, 5
H, W           = 360, 600
TILE_H, TILE_W = H // GRID_R, W // GRID_C   # 120 x 120
N_TILES        = GRID_R * GRID_C            # 15

# ---- Trọng số tín hiệu (khởi tạo; sẽ học ONLINE trong TRAIN) ----
W_INIT = {
    "alpha": 1.00,   # Lab color MSE
    "beta":  0.35,   # gradient MSE
    "gamma": 0.80,   # 1 - SSIM
    "ori":   0.10,   # HOG chi2
    "lbp":   0.10,   # LBP chi2
    "prior_w": 0.10  # trọng số location prior theo hàng (trên/giữa/dưới)
}
W_BOUNDS = {
    "alpha": (0.0, 2.0),
    "beta":  (0.0, 1.5),
    "gamma": (0.0, 1.5),
    "ori":   (0.0, 1.0),
    "lbp":   (0.0, 1.0),
    "prior_w": (0.0, 0.4),
}

# ---- Fine-tune per-image (TRAIN ONLY) — grid nhỏ để không chạy quá lâu ----
TUNE_BAND    = [5, 6]                     # dải biên khi so khớp cạnh
TUNE_PRIOR_W = [0.00, 0.10]               # quét nhẹ prior_w
TUNE_DW = {                               
    # Quét nhẹ quanh trọng số hiện tại (alpha giữ nguyên cho ổn định)
    "alpha": [0.0],
    "beta":  [-0.05, 0.0, +0.05],
    "gamma": [-0.10, 0.0, +0.10],
    "ori":   [0.0, +0.05],
    "lbp":   [0.0, +0.05],
}

# ---- Online learning (TRAIN ONLY) — unsupervised metric learning ----
LEARN_CFG = {
    "negatives_per_pos": 2,   # số negative cho mỗi positive edge
    "margin": 0.06,           # biên hinge (mức "gắt")
    "lr": 0.04,               # learning rate
    "ema": 0.85,              # làm mượt tham số (ổn định)
    "max_pairs_per_image": 40,# trần số cặp train/ảnh
    "band_for_learning": 6    # band dùng khi trích feature cho học
}

# ---- Auto-budget mặc định (2h); bạn có presets 1h/2h/3h ở cuối Cell ----
TOTAL_TRAIN_IMAGES = 0
try:
    TOTAL_TRAIN_IMAGES = len(glob.glob(os.path.join(TRAIN_TASK1_DIR, "*.png"))) \
                       + len(glob.glob(os.path.join(TRAIN_TASK2_DIR, "*.png")))
except Exception:
    TOTAL_TRAIN_IMAGES = 0
TOTAL_TRAIN_IMAGES         = max(TOTAL_TRAIN_IMAGES, 1)  # tránh chia 0
TOTAL_TRAIN_SECONDS        = 2 * 3600                     # mặc định 2 giờ
PER_IMAGE_BUDGET_SEC       = int(max(20, min(80, (TOTAL_TRAIN_SECONDS*0.85)/TOTAL_TRAIN_IMAGES)))
_EST_TRIAL_SEC             = 3.5                          # ước lượng 1 trial
MAX_PARAM_TRIALS_PER_IMAGE = int(max(6, min(14, PER_IMAGE_BUDGET_SEC/_EST_TRIAL_SEC)))
TIME_BUDGET_PER_IMAGE_SEC  = int(PER_IMAGE_BUDGET_SEC)

# ---- File IO ----
PUBLIC_CSV   = os.path.join(OUTPUT_DIR, "public_output.csv")
PRIVATE_CSV  = os.path.join(OUTPUT_DIR, "private_output.csv")
MODEL_PATH   = os.path.join(OUTPUT_DIR, "trained_model.json")
RUN_LOG      = os.path.join(OUTPUT_DIR, "run.log")
SUBMISSION   = os.path.join(OUTPUT_DIR, "submission.zip")

# Optional hints CSV cho Task1 (nếu có)
HINTS_TRAIN1 = os.path.join(OUTPUT_DIR, "task1_hints.csv")
HINTS_PUBLIC = os.path.join(OUTPUT_DIR, "public_task1_hints.csv")
HINTS_PRIVATE= os.path.join(OUTPUT_DIR, "private_task1_hints.csv")

# ---- Reproducibility ----
random.seed(42)
np.random.seed(42)

# ---- Logger (file + stdout) ----
for h in list(logging.root.handlers):
    logging.root.removeHandler(h)
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    handlers=[logging.FileHandler(RUN_LOG, mode="w", encoding="utf-8"),
              logging.StreamHandler(sys.stdout)]
)
logger = logging.getLogger("VOAI2025_Task2")
logger.info(f"BASE_DIR = {BASE_DIR}")
logger.info("Folders ready. Logging initialized.")
logger.info(f"[DEFAULT BUDGET] train_imgs={TOTAL_TRAIN_IMAGES} | per_image={PER_IMAGE_BUDGET_SEC}s "
            f"| max_trials={MAX_PARAM_TRIALS_PER_IMAGE} | learn_cfg={LEARN_CFG}")

# ============================================================
# ⏱️ TIME PRESETS (1h/2h/3h) — dễ chỉnh thời lượng TRAIN
#  Cách dùng: đổi TRAIN_PRESET thành "1h" | "2h" | "3h"
# ============================================================
TRAIN_PRESET = "2h"  # ← đổi ở đây nếu muốn 1h/3h nhanh chóng

PRESET_CONFIGS = {
    "1h": {
        "TUNE_BAND": [6],
        "TUNE_PRIOR_W": [0.10],
        "TUNE_DW": {"alpha":[0.0],"beta":[-0.05,0.0,0.05],"gamma":[-0.10,0.0,0.10],"ori":[0.0],"lbp":[0.0]},
        "NEG": 2, "MAX_PAIRS": 30, "EMA": 0.85,
        "EST_TRIAL_SEC": 3.5, "TRIALS_RANGE": (6,10), "PER_IMAGE_CAP": (18,35),
        "TOTAL_TRAIN_SECONDS": 1*3600,
    },
    "2h": {
        "TUNE_BAND": [5,6],
        "TUNE_PRIOR_W": [0.00,0.10],
        "TUNE_DW": {"alpha":[0.0],"beta":[-0.05,0.0,0.05],"gamma":[-0.10,0.0,0.10],"ori":[0.0,0.05],"lbp":[0.0,0.05]},
        "NEG": 2, "MAX_PAIRS": 40, "EMA": 0.85,
        "EST_TRIAL_SEC": 3.5, "TRIALS_RANGE": (8,14), "PER_IMAGE_CAP": (35,70),
        "TOTAL_TRAIN_SECONDS": 2*3600,
    },
    "3h": {
        "TUNE_BAND": [5,6,7],
        "TUNE_PRIOR_W": [0.00,0.05,0.10,0.15],
        "TUNE_DW": {"alpha":[0.0],"beta":[-0.10,-0.05,0.0,0.05,0.10],"gamma":[-0.15,-0.10,0.0,0.10,0.15],"ori":[0.0,0.05],"lbp":[0.0,0.05]},
        "NEG": 3, "MAX_PAIRS": 60, "EMA": 0.80,
        "EST_TRIAL_SEC": 3.2, "TRIALS_RANGE": (10,18), "PER_IMAGE_CAP": (60,95),
        "TOTAL_TRAIN_SECONDS": 3*3600,
    },
}

def _apply_time_preset(preset):
    """Áp cấu hình thời gian train (1h/2h/3h) một cách an toàn."""
    global TUNE_BAND, TUNE_PRIOR_W, TUNE_DW
    global LEARN_CFG, MAX_PARAM_TRIALS_PER_IMAGE, TIME_BUDGET_PER_IMAGE_SEC
    global TOTAL_TRAIN_SECONDS, PER_IMAGE_BUDGET_SEC, _EST_TRIAL_SEC

    if preset not in PRESET_CONFIGS:
        print(f"[WARN] Unknown preset: {preset}. Keep defaults.")
        return

    cfg = PRESET_CONFIGS[preset]
    TUNE_BAND, TUNE_PRIOR_W, TUNE_DW = cfg["TUNE_BAND"], cfg["TUNE_PRIOR_W"], cfg["TUNE_DW"]
    LEARN_CFG["negatives_per_pos"]   = cfg["NEG"]
    LEARN_CFG["max_pairs_per_image"] = cfg["MAX_PAIRS"]
    LEARN_CFG["ema"]                 = cfg["EMA"]

    TOTAL_TRAIN_SECONDS = cfg["TOTAL_TRAIN_SECONDS"]
    _EST_TRIAL_SEC      = cfg["EST_TRIAL_SEC"]

    # Tự ước lượng ngân sách/ảnh theo số ảnh train (dành ~85% cho solve+tuning)
    total_imgs = 0
    try:
        total_imgs = len(glob.glob(os.path.join(TRAIN_TASK1_DIR, "*.png"))) \
                   + len(glob.glob(os.path.join(TRAIN_TASK2_DIR, "*.png")))
    except Exception:
        total_imgs = 0
    total_imgs = max(total_imgs, 1)

    raw_budget = (TOTAL_TRAIN_SECONDS * 0.85) / total_imgs
    lo, hi     = cfg["PER_IMAGE_CAP"]
    PER_IMAGE_BUDGET_SEC = int(max(lo, min(hi, raw_budget)))
    lo_t, hi_t = cfg["TRIALS_RANGE"]
    MAX_PARAM_TRIALS_PER_IMAGE = int(max(lo_t, min(hi_t, PER_IMAGE_BUDGET_SEC / _EST_TRIAL_SEC)))
    TIME_BUDGET_PER_IMAGE_SEC  = int(PER_IMAGE_BUDGET_SEC)

    print(f"[PRESET] {preset} | train_imgs={total_imgs} | per_image={PER_IMAGE_BUDGET_SEC}s | "
          f"max_trials={MAX_PARAM_TRIALS_PER_IMAGE} | learn_cfg={LEARN_CFG}")

_apply_time_preset(TRAIN_PRESET)
print("✅ Cell 0 ready.")


2025-10-29 04:54:55,477 | INFO | BASE_DIR = /home/dammanhdungvn/learn_ml/project
2025-10-29 04:54:55,478 | INFO | Folders ready. Logging initialized.
2025-10-29 04:54:55,479 | INFO | [DEFAULT BUDGET] train_imgs=16 | per_image=80s | max_trials=14 | learn_cfg={'negatives_per_pos': 2, 'margin': 0.06, 'lr': 0.04, 'ema': 0.85, 'max_pairs_per_image': 40, 'band_for_learning': 6}
[PRESET] 2h | train_imgs=16 | per_image=70s | max_trials=14 | learn_cfg={'negatives_per_pos': 2, 'margin': 0.06, 'lr': 0.04, 'ema': 0.85, 'max_pairs_per_image': 40, 'band_for_learning': 6}
✅ Cell 0 ready.


In [2]:
# ============================================================
# Cell 1 — IO HELPERS
# ------------------------------------------------------------
# - list_images: chỉ liệt kê *.png (đúng đề bài).
# - load_image_rgb: đọc RGB + assert đúng 600x360.
# - load_hints_csv: nếu có file hint cho Task1 (0,0).
# ============================================================

def list_images(folder, exts=(".png",)):
    try:
        files = [p for p in glob.glob(os.path.join(folder, "*")) if p.lower().endswith(exts)]
        files.sort()
        return files
    except Exception as e:
        logger.error(f"Cannot list images in {folder}: {e}")
        return []

def load_image_rgb(path):
    bgr = cv2.imread(path, cv2.IMREAD_COLOR)
    if bgr is None:
        raise FileNotFoundError(f"Cannot read image: {path}")
    h, w = bgr.shape[:2]
    if (h, w) != (H, W):
        raise AssertionError(f"Image must be {H}x{W}, got {(h,w)} @ {os.path.basename(path)}")
    return cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)

def load_hints_csv(path):
    """
    CSV (optional): image_filename, top_left_piece_index
    Dùng cho Task1 nếu bạn có chỉ số mảnh (0,0).
    """
    if not os.path.isfile(path):
        logger.warning(f"Hint file not found (optional): {path}")
        return {}
    try:
        df = pd.read_csv(path)
        mp = {str(row["image_filename"]).strip(): int(row["top_left_piece_index"]) for _, row in df.iterrows()}
        logger.info(f"Loaded {len(mp)} hints from {path}.")
        return mp
    except Exception as e:
        logger.error(f"Failed to read hints CSV {path}: {e}")
        return {}

print("✅ Cell 1 ready.")


✅ Cell 1 ready.


In [3]:
# ============================================================
# Cell 2 — TILING & ASSEMBLE
# ------------------------------------------------------------
# - Tách ảnh xáo trộn thành 15 mảnh (row-major).
# - Lắp lại theo hoán vị (perm) cho visualization & export.
# ============================================================

def tiles_from_shuffled(rgb):
    tiles = []
    for r in range(GRID_R):
        for c in range(GRID_C):
            y0, y1 = r*TILE_H, (r+1)*TILE_H
            x0, x1 = c*TILE_W, (c+1)*TILE_W
            tiles.append(rgb[y0:y1, x0:x1].copy())
    return tiles  # list len=15

def assemble_by_perm(tiles, perm):
    canvas = np.zeros((H, W, 3), dtype=np.uint8)
    k = 0
    for r in range(GRID_R):
        for c in range(GRID_C):
            y0, y1 = r*TILE_H, (r+1)*TILE_H
            x0, x1 = c*TILE_W, (c+1)*TILE_W
            canvas[y0:y1, x0:x1] = tiles[perm[k]]
            k += 1
    return canvas

print("✅ Cell 2 ready.")


✅ Cell 2 ready.


In [4]:
# ============================================================
# Cell 3 — EDGE FEATURES (5 cues)
# ------------------------------------------------------------
# Tín hiệu cho cạnh i->j theo hướng d ∈ {L,R,U,D}:
#   f = [ MSE_Lab, MSE_grad, (1-SSIM_L), HOG_chi2, LBP_chi2 ]
# Cost cạnh = w · f  (w học online ở TRAIN)
# ============================================================

from skimage.color import rgb2lab
from skimage.metrics import structural_similarity as ssim
from skimage.feature import local_binary_pattern

DIRS = ['L','R','U','D']
OPP  = {'L':'R','R':'L','U':'D','D':'U'}

def _lab01(tile_u8):
    return rgb2lab(tile_u8.astype(np.float32)/255.0)

def _band(img, side, band):
    if side == 'L': return img[:, :band, :]
    if side == 'R': return img[:, -band:, :]
    if side == 'U': return img[:band, :, :]
    if side == 'D': return img[-band:, :, :]
    raise ValueError(side)

def _grad_x(img):
    gx = img[:,1:,:] - img[:,:-1,:]
    gx = np.pad(gx, ((0,0),(0,1),(0,0)), mode='edge')
    return gx

def _grad_y(img):
    gy = img[1:,:,:] - img[:-1,:,:]
    gy = np.pad(gy, ((0,1),(0,0),(0,0)), mode='edge')
    return gy

def _normalize_band(arr):
    mu = arr.mean(axis=(0,1), keepdims=True)
    sd = arr.std(axis=(0,1), keepdims=True) + 1e-6
    return (arr - mu)/sd

def _ori_hist(lab, side, band, nbins=9):
    # HOG đơn giản từ kênh L
    gx = _grad_x(lab)[...,0]
    gy = _grad_y(lab)[...,0]
    mag = np.sqrt(gx**2 + gy**2)
    ori = np.arctan2(np.abs(gy), np.abs(gx))  # 0..pi/2

    def band2(arr):
        if side=='L': return arr[:, :band]
        if side=='R': return arr[:, -band:]
        if side=='U': return arr[:band, :]
        if side=='D': return arr[-band:, :]
    mag_b = band2(mag); ori_b = band2(ori)

    hist, _ = np.histogram(ori_b, bins=nbins, range=(0, np.pi/2), weights=mag_b)
    hist = hist.astype(np.float32); hist /= (hist.sum() + 1e-6)
    return hist

def _lbp_hist(lab, side, band, P=8, R=1.0):
    L = lab[...,0]
    lbp = local_binary_pattern(L, P=P, R=R, method='uniform')
    n_bins = P + 2

    def band2(arr):
        if side=='L': return arr[:, :band]
        if side=='R': return arr[:, -band:]
        if side=='U': return arr[:band, :]
        if side=='D': return arr[-band:, :]
    lbp_b = band2(lbp)

    hist, _ = np.histogram(lbp_b, bins=np.arange(0, n_bins+1), range=(0, n_bins))
    hist = hist.astype(np.float32); hist /= (hist.sum() + 1e-6)
    return hist

def _chi2(p, q):
    return 0.5 * np.sum(((p - q)**2) / (p + q + 1e-8))

def precompute_feats(tiles):
    """Tiền tính Lab & gradient cho từng mảnh (tăng tốc khi so khớp)."""
    feats = []
    for t in tiles:
        lab = _lab01(t)
        gx  = _grad_x(lab)
        gy  = _grad_y(lab)
        feats.append((lab, gx, gy))
    return feats

def edge_feature_vec(fA, fB, direction, band):
    """Trả về vector 5-dim f cho cạnh fA->fB theo hướng 'direction'."""
    labA, gxA, gyA = fA
    labB, gxB, gyB = fB

    if direction == 'R':
        a_c, b_c = _band(labA,'R',band), _band(labB,'L',band)
        a_gx, b_gx = gxA[:, -band:, :], gxB[:, :band, :]
        a_gy, b_gy = gyA[:, -band:, :], gyB[:, :band, :]
    elif direction == 'L':
        a_c, b_c = _band(labA,'L',band), _band(labB,'R',band)
        a_gx, b_gx = gxA[:, :band, :],  gxB[:, -band:, :]
        a_gy, b_gy = gyA[:, :band, :],  gyB[:, -band:, :]
    elif direction == 'D':
        a_c, b_c = _band(labA,'D',band), _band(labB,'U',band)
        a_gx, b_gx = gxA[-band:, :, :], gxB[:band, :, :]
        a_gy, b_gy = gyA[-band:, :, :], gyB[:band, :, :]
    else:  # 'U'
        a_c, b_c = _band(labA,'U',band), _band(labB,'D',band)
        a_gx, b_gx = gxA[:band, :, :],  gxB[-band:, :, :]
        a_gy, b_gy = gyA[:band, :, :],  gyB[-band:, :, :]

    # 1) MSE màu (Lab)
    a_c_n, b_c_n = _normalize_band(a_c), _normalize_band(b_c)
    mse_c = float(np.mean((a_c_n - b_c_n)**2))

    # 2) MSE gradient (năng lượng)
    a_g = np.sqrt(a_gx**2 + a_gy**2)
    b_g = np.sqrt(b_gx**2 + b_gy**2)
    a_g_n, b_g_n = _normalize_band(a_g), _normalize_band(b_g)
    mse_g = float(np.mean((a_g_n - b_g_n)**2))

    # 3) (1 - SSIM) trên kênh L
    a_l = a_c[...,0]; b_l = b_c[...,0]
    a_l_n = (a_l - a_l.mean())/(a_l.std()+1e-6)
    b_l_n = (b_l - b_l.mean())/(b_l.std()+1e-6)
    try:
        ssim_val = ssim(a_l_n, b_l_n, data_range=(a_l_n.max()-a_l_n.min()))
    except Exception:
        ssim_val = 0.0
    cost_ssim = 1.0 - float(ssim_val)

    # 4) HOG chi2, 5) LBP chi2
    sideA = {'R':'R','L':'L','D':'D','U':'U'}[direction]
    sideB = {'R':'L','L':'R','D':'U','U':'D'}[direction]
    chi2_ori = _chi2(_ori_hist(labA, sideA, band), _ori_hist(labB, sideB, band))
    chi2_lbp = _chi2(_lbp_hist(labA, sideA, band), _lbp_hist(labB, sideB, band))

    return np.array([mse_c, mse_g, cost_ssim, chi2_ori, chi2_lbp], dtype=np.float32)

print("✅ Cell 3 ready.")


✅ Cell 3 ready.


In [5]:
# ============================================================
# Cell 4 — COST & MATRICES
# ------------------------------------------------------------
# - dot_cost: cost cạnh từ feat 5-dim và trọng số w.
# - precompute_costs_and_orders: ma trận cost & danh sách gợi ý theo từng hướng.
# ============================================================

def dot_cost(feat5, w):
    return (w["alpha"]*feat5[0] +
            w["beta"] *feat5[1] +
            w["gamma"]*feat5[2] +
            w["ori"]  *feat5[3] +
            w["lbp"]  *feat5[4])

def precompute_costs_and_orders(feats, band, w):
    n = len(feats)
    costs = {d: np.full((n, n), np.inf, dtype=np.float32) for d in DIRS}
    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            for d in DIRS:
                f5 = edge_feature_vec(feats[i], feats[j], d, band)
                costs[d][i, j] = dot_cost(f5, w)
    # Mỗi (i,d) có list các j theo thứ tự cost tăng dần (loại i)
    orders = {d: [] for d in DIRS}
    for d in DIRS:
        for i in range(n):
            idx = np.argsort(costs[d][i, :]).tolist()
            orders[d].append([k for k in idx if k != i])
    return costs, orders

print("✅ Cell 4 ready.")


✅ Cell 4 ready.


In [6]:
# ============================================================
# Cell 5 — LOCATION PRIOR (mềm theo hàng ảnh)
# ------------------------------------------------------------
# Ưu tiên hàng 0 (trên): b trung bình nhỏ; hàng 2 (dưới): b lớn hơn.
# Trả ma trận prior[piece, position], dùng như 1 thành phần cost mềm.
# ============================================================

from skimage.color import rgb2lab

def compute_location_prior(tiles):
    n = len(tiles)
    pri = np.zeros((n, GRID_R*GRID_C), dtype=np.float32)
    labs = [rgb2lab(t.astype(np.float32)/255.0) for t in tiles]
    means = [(L[...,0].mean(), L[...,1].mean(), L[...,2].mean()) for L in labs]  # (L,a,b)

    targets_b = [-5.0,  5.0, 20.0]  # row 0→1→2
    targets_a = [ 0.0,  0.0, -5.0]

    for p, (L, a, b) in enumerate(means):
        for pos in range(GRID_R*GRID_C):
            r = pos // GRID_C
            tb, ta = targets_b[r], targets_a[r]
            pri[p, pos] = ((b - tb)/20.0)**2 + 0.5*((a - ta)/15.0)**2
    return pri

print("✅ Cell 5 ready.")


✅ Cell 5 ready.


In [7]:
# ============================================================
# Cell 6 — HEURISTIC + A*
# ------------------------------------------------------------
# f = g + h với h là lower bound:
# - Tổng biên mở (R/D) còn lại.
# - Min prior cho các vị trí chưa đặt.
# ============================================================

def right_index(idx):
    r, c = divmod(idx, GRID_C)
    return idx + 1 if c + 1 < GRID_C else None

def down_index(idx):
    r, c = divmod(idx, GRID_C)
    return idx + GRID_C if r + 1 < GRID_R else None

def min_unused_for(i, d, costs, orders, used_mask):
    """Chi phí thấp nhất i->j theo hướng d với j chưa dùng."""
    for j in orders[d][i]:
        if not ((used_mask >> j) & 1):
            return float(costs[d][i, j])
    return 0.0

def heuristic_with_prior(placement, used_mask, costs, orders, prior, prior_w):
    t = len(placement); h = 0.0
    # LB cho các cạnh mở
    for i in range(t):
        p = placement[i]
        jR = right_index(i)
        if jR is not None and jR >= t:
            h += min_unused_for(p, 'R', costs, orders, used_mask)
        jD = down_index(i)
        if jD is not None and jD >= t:
            h += min_unused_for(p, 'D', costs, orders, used_mask)
    # LB cho prior các vị trí trống
    remaining = list(range(t, GRID_R*GRID_C))
    unused   = [j for j in range(prior.shape[0]) if not ((used_mask >> j) & 1)]
    for pos in remaining:
        if not unused: break
        minp = min(prior[j, pos] for j in unused)
        h += prior_w * minp
    return h

def solve_astar_with_prior(tiles, costs, orders, prior, prior_w, initial_piece=None, log_prefix=""):
    n = len(tiles)
    frontier = []
    best_g = {}  # (idx, used_mask) -> best g đã gặp

    def push_state(placement, used_mask, g):
        h = heuristic_with_prior(placement, used_mask, costs, orders, prior, prior_w)
        heapq.heappush(frontier, (g+h, g, placement, used_mask))

    # Seed (Task1 có hint; Task2 thì thử mọi mảnh làm (0,0))
    if initial_piece is not None:
        used = (1 << initial_piece)
        placement = [initial_piece]
        g0 = prior_w * prior[initial_piece, 0]
        push_state(placement, used, g0)
        best_g[(1, used)] = g0
    else:
        for p in range(n):
            used = (1 << p)
            placement = [p]
            g0 = prior_w * prior[p, 0]
            push_state(placement, used, g0)
            best_g[(1, used)] = g0

    expansions = 0
    while frontier:
        f, g, placement, used = heapq.heappop(frontier)
        idx = len(placement)
        if idx == n:
            logger.info(f"{log_prefix}A* done | expansions={expansions} | best_cost={g:.6f}")
            return placement, g

        if best_g.get((idx, used), float('inf')) < g - 1e-12:
            continue

        r, c = divmod(idx, GRID_C)
        left_idx = idx - 1 if c - 1 >= 0 else None
        up_idx   = idx - GRID_C if r - 1 >= 0 else None

        # Ứng viên ưu tiên theo cost cục bộ + prior vị trí
        candidates = [p for p in range(n) if not ((used >> p) & 1)]
        def local_inc(p):
            inc = 0.0
            if left_idx is not None: inc += float(costs['R'][placement[left_idx], p])
            if up_idx   is not None: inc += float(costs['D'][placement[up_idx], p])
            inc += prior_w * prior[p, idx]
            return inc
        candidates.sort(key=local_inc)

        for p in candidates:
            add = 0.0
            if left_idx is not None: add += float(costs['R'][placement[left_idx], p])
            if up_idx   is not None: add += float(costs['D'][placement[up_idx], p])
            add += prior_w * prior[p, idx]

            used_new = used | (1 << p)
            placement_new = placement + [p]
            g_new = g + add
            key = (idx + 1, used_new)

            if g_new + 1e-12 < best_g.get(key, float('inf')):
                best_g[key] = g_new
                push_state(placement_new, used_new, g_new)

        expansions += 1
        if expansions % 5000 == 0:
            logger.info(f"{log_prefix}expansions={expansions}, frontier={len(frontier)}")

    raise RuntimeError(f"{log_prefix}No solution found.")

print("✅ Cell 6 ready.")


✅ Cell 6 ready.


In [8]:
# ============================================================
# Cell 7 — OBJECTIVE & 2-OPT LOCAL REPAIR
# ------------------------------------------------------------
# Sau A*, thử hoán đổi cặp mảnh (2-opt) để hạ cost thêm chút
# (chữa lỗi nhỏ do heuristic không hoàn hảo).
# ============================================================

def right_neighbor_index(pos):
    r, c = divmod(pos, GRID_C)
    return pos + 1 if c + 1 < GRID_C else None

def down_neighbor_index(pos):
    r, c = divmod(pos, GRID_C)
    return pos + GRID_C if r + 1 < GRID_R else None

def total_cost_for_perm(perm, costs, prior, prior_w):
    tot = 0.0
    for pos, piece in enumerate(perm):
        rn = right_neighbor_index(pos)
        dn = down_neighbor_index(pos)
        if rn is not None:
            tot += float(costs['R'][piece, perm[rn]])
        if dn is not None:
            tot += float(costs['D'][piece, perm[dn]])
        tot += prior_w * prior[piece, pos]
    return float(tot)

def local_improve_2opt(perm, costs, prior, prior_w, max_iters=200, patience=40):
    best = perm[:]
    best_cost = total_cost_for_perm(best, costs, prior, prior_w)
    no_improve = 0
    n = len(perm)

    while no_improve < patience and max_iters > 0:
        improved = False
        for i in range(n-1):
            for j in range(i+1, n):
                cand = best[:]
                cand[i], cand[j] = cand[j], cand[i]
                c = total_cost_for_perm(cand, costs, prior, prior_w)
                if c + 1e-9 < best_cost:
                    best, best_cost = cand, c
                    improved = True
                    break
            if improved: break
        no_improve = 0 if improved else (no_improve + 1)
        max_iters -= 1
    return best, best_cost

print("✅ Cell 7 ready.")


✅ Cell 7 ready.


In [9]:
# ============================================================
# Cell 8 — SOLVE ONE IMAGE WITH (w, band, prior_w)
# ------------------------------------------------------------
# - Tách mảnh → tiền tính đặc trưng → cost ma trận → A* → 2-opt.
# - Trả perm, cost, ảnh ghép & các cache (tiles, feats, costs, prior).
# ============================================================

def solve_one_image_with_params(path, w, band, prior_w, initial_piece=None, do_local_improve=True):
    rgb = load_image_rgb(path)
    tiles = tiles_from_shuffled(rgb)
    feats = precompute_feats(tiles)
    costs, orders = precompute_costs_and_orders(feats, band, w)
    prior = compute_location_prior(tiles)

    perm, best_cost = solve_astar_with_prior(
        tiles, costs, orders, prior,
        prior_w=prior_w,
        initial_piece=initial_piece,
        log_prefix=f"{os.path.basename(path)} | "
    )
    if do_local_improve:
        perm, best_cost = local_improve_2opt(perm, costs, prior, prior_w, max_iters=120, patience=30)
    vis = assemble_by_perm(tiles, perm)
    return perm, best_cost, vis, tiles, feats, costs, prior

print("✅ Cell 8 ready.")


✅ Cell 8 ready.


In [10]:
# ============================================================
# Cell 9 — AUTO FINE-TUNING (TRAIN ONLY)
# ------------------------------------------------------------
# Sinh tập tham số (w, band, prior_w) quanh w hiện tại, thử có
# giới hạn thời gian/lượt, chọn cấu hình tốt nhất cho từng ảnh.
# ============================================================

def clip_weight(name, val):
    lo, hi = W_BOUNDS[name]
    return float(np.clip(val, lo, hi))

def generate_param_candidates(w_base):
    """Sinh danh sách (w, band, prior_w) quanh w_base theo TUNE_*."""
    cands = []
    for band in TUNE_BAND:
        for pw in TUNE_PRIOR_W:
            for db in TUNE_DW["beta"]:
                for dg in TUNE_DW["gamma"]:
                    for do in TUNE_DW["ori"]:
                        for dl in TUNE_DW["lbp"]:
                            w = dict(w_base)
                            w["beta"]  = clip_weight("beta",  w["beta"]  + db)
                            w["gamma"] = clip_weight("gamma", w["gamma"] + dg)
                            w["ori"]   = clip_weight("ori",   w["ori"]   + do)
                            w["lbp"]   = clip_weight("lbp",   w["lbp"]   + dl)
                            cands.append( (w, band, clip_weight("prior_w", pw)) )
    random.Random(0).shuffle(cands)
    return cands

def choose_best_for_image_with_tuning(path, w_base, initial_piece=None):
    """Chọn cấu hình tốt nhất cho 1 ảnh (theo cost) trong ngân sách cho phép."""
    param_sets = generate_param_candidates(w_base)
    best = {"perm": None, "cost": float("inf"), "w": None, "band": None, "prior_w": None, "vis": None,
            "tiles": None, "feats": None, "costs": None, "prior": None}
    start = time.time()
    trials = 0

    for (w, band, pw) in param_sets:
        if trials >= MAX_PARAM_TRIALS_PER_IMAGE:
            logger.info(f"{os.path.basename(path)} | stop by trial cap: {trials}")
            break
        if time.time() - start > TIME_BUDGET_PER_IMAGE_SEC:
            logger.info(f"{os.path.basename(path)} | stop by time budget: {TIME_BUDGET_PER_IMAGE_SEC}s")
            break
        try:
            perm, cost, vis, tiles, feats, costs, prior = solve_one_image_with_params(
                path, w, band, pw, initial_piece=initial_piece, do_local_improve=True
            )
            trials += 1
            if cost < best["cost"]:
                best.update({"perm":perm,"cost":cost,"w":w,"band":band,"prior_w":pw,"vis":vis,
                             "tiles":tiles,"feats":feats,"costs":costs,"prior":prior})
        except Exception as e:
            logger.error(f"{os.path.basename(path)} | params (band={band}, pw={pw}, w={w}) failed: {e}")

    if best["perm"] is None:
        # Fallback an toàn nếu toàn bộ thử lỗi
        perm, cost, vis, tiles, feats, costs, prior = solve_one_image_with_params(
            path, w_base, band=6, prior_w=w_base["prior_w"], initial_piece=initial_piece, do_local_improve=True
        )
        best.update({"perm":perm,"cost":cost,"w":w_base,"band":6,"prior_w":w_base["prior_w"],
                     "vis":vis,"tiles":tiles,"feats":feats,"costs":costs,"prior":prior})

    logger.info(f"{os.path.basename(path)} | best_cost={best['cost']:.6f} | band={best['band']} | "
                f"prior_w={best['prior_w']} | w={best['w']}")
    return best

print("✅ Cell 9 ready.")


✅ Cell 9 ready.


In [11]:
# ============================================================
# Cell 10 — UNSUPERVISED ONLINE LEARNING (TRAIN ONLY)
# ------------------------------------------------------------
# - Pseudo-label cạnh dương từ perm tốt nhất; âm = top ứng viên còn lại.
# - Hinge loss: max(0, m + w·f_pos - w·f_neg)
# - Projected GD + EMA smoothing.
# ============================================================

class AdaptiveCueWeights:
    def __init__(self, w0, bounds, cfg):
        self.w = dict(w0)   # alpha..lbp, prior_w (prior_w cập nhật nhẹ qua EMA)
        self.bounds = bounds
        self.cfg = cfg

    def _proj(self):
        for k, (lo, hi) in self.bounds.items():
            self.w[k] = float(np.clip(self.w[k], lo, hi))

    def _feat(self, feats, i, j, d, band):
        return edge_feature_vec(feats[i], feats[j], d, band)

    def update_with_pseudo(self, best, costs):
        """Cập nhật self.w (alpha..lbp) từ pseudo edges trong best perm."""
        perm = best["perm"]; feats = best["feats"]; band = self.cfg["band_for_learning"]
        if feats is None or perm is None:
            return

        # Duyệt positive edges theo grid (R, D)
        pos_edges = []
        for pos, p in enumerate(perm):
            r, c = divmod(pos, GRID_C)
            if c + 1 < GRID_C:
                pos_edges.append( (p, perm[pos+1], 'R') )
            if r + 1 < GRID_R:
                pos_edges.append( (p, perm[pos+GRID_C], 'D') )

        neg_per_pos = self.cfg["negatives_per_pos"]
        pairs = []
        for (i, j_true, d) in pos_edges:
            row = costs[d][i,:].copy()
            row[i] = np.inf
            cand = np.argsort(row).tolist()
            negs = [k for k in cand if k != j_true][:neg_per_pos]
            if not negs:
                continue
            f_pos = self._feat(feats, i, j_true, d, band)
            for j_neg in negs:
                f_neg = self._feat(feats, i, j_neg, d, band)
                pairs.append( (f_pos, f_neg) )
            if len(pairs) >= self.cfg["max_pairs_per_image"]:
                break

        if not pairs:
            return

        m  = self.cfg["margin"]
        lr = self.cfg["lr"]
        grad = {"alpha":0.0,"beta":0.0,"gamma":0.0,"ori":0.0,"lbp":0.0}
        count = 0

        for f_pos, f_neg in pairs:
            s_pos = (self.w["alpha"]*f_pos[0] + self.w["beta"]*f_pos[1] + self.w["gamma"]*f_pos[2] +
                     self.w["ori"]*f_pos[3]   + self.w["lbp"]*f_pos[4])
            s_neg = (self.w["alpha"]*f_neg[0] + self.w["beta"]*f_neg[1] + self.w["gamma"]*f_neg[2] +
                     self.w["ori"]*f_neg[3]   + self.w["lbp"]*f_neg[4])
            loss = m + s_pos - s_neg
            if loss > 0:
                grad["alpha"] += (f_pos[0] - f_neg[0])
                grad["beta"]  += (f_pos[1] - f_neg[1])
                grad["gamma"] += (f_pos[2] - f_neg[2])
                grad["ori"]   += (f_pos[3] - f_neg[3])
                grad["lbp"]   += (f_pos[4] - f_neg[4])
                count += 1

        if count > 0:
            scale = lr / count
            w_new = dict(self.w)
            for k in ["alpha","beta","gamma","ori","lbp"]:
                w_new[k] = self.w[k] - scale * grad[k]
                lo, hi = self.bounds[k]; w_new[k] = float(np.clip(w_new[k], lo, hi))
            # EMA smoothing
            ema = self.cfg["ema"]
            for k in ["alpha","beta","gamma","ori","lbp"]:
                self.w[k] = ema*self.w[k] + (1-ema)*w_new[k]
            self._proj()

    def get_w(self):
        return dict(self.w)

print("✅ Cell 10 ready.")


✅ Cell 10 ready.


In [12]:
# ============================================================
# Cell 11 — TRAIN PIPELINE
# ------------------------------------------------------------
# - Mỗi ảnh train:
#   + (Nếu có) hint cho task1 -> cố định (0,0).
#   + Auto tuning (grid nhỏ, có giới hạn thời gian/lượt).
#   + Lưu ảnh đã ghép vào images/sloved/task{1,2}.
#   + Online learning: cập nhật w (alpha..lbp) từ pseudo-edges.
# - Cuối cùng: chọn band/prior_w toàn cục theo "mode" và FREEZE.
#   => Lưu model JSON: weights (alpha..lbp, prior_w), band, prior_w.
# ============================================================

def run_train_split(train_dir, sloved_dir, learner, hints_csv=None):
    os.makedirs(sloved_dir, exist_ok=True)
    files = list_images(train_dir)
    if not files:
        logger.warning(f"[TRAIN] Empty folder: {train_dir}")
    hints = load_hints_csv(hints_csv) if hints_csv else {}
    records = []  # lưu (band, prior_w) tốt nhất của từng ảnh

    for path in tqdm(files, desc=f"TRAIN on {os.path.basename(train_dir)}"):
        fname = os.path.basename(path)
        try:
            hint = hints.get(fname, None)
            w_base = learner.get_w()
            best = choose_best_for_image_with_tuning(path, w_base, initial_piece=hint)

            # Lưu ảnh đã ghép
            out_path = os.path.join(sloved_dir, fname)
            ok = cv2.imwrite(out_path, cv2.cvtColor(best["vis"], cv2.COLOR_RGB2BGR))
            if not ok:
                logger.warning(f"Failed to save solved image: {out_path}")

            logger.info(f"[TRAIN:{fname}] cost={best['cost']:.6f} | band={best['band']} | prior_w={best['prior_w']} | w={best['w']}")

            # Học online (chỉ alpha..lbp). prior_w cập nhật nhẹ qua EMA dưới đây
            learner.update_with_pseudo(best, best["costs"])

            # EMA nhẹ cho prior_w dựa vào best
            learner.w["prior_w"] = float(np.clip(
                LEARN_CFG["ema"] * learner.w["prior_w"] + (1-LEARN_CFG["ema"])*best["prior_w"],
                *W_BOUNDS["prior_w"]
            ))

            records.append((best["band"], best["prior_w"]))
        except Exception:
            logger.exception(f"[TRAIN:{fname}] error")
    return records

def mode_select(values):
    if not values:
        return None
    vals, cnts = np.unique(values, return_counts=True)
    return vals[np.argmax(cnts)]

# ---- Thực thi TRAIN ----
logger.info("===== TRAINING PHASE START =====")
learner = AdaptiveCueWeights(W_INIT, W_BOUNDS, LEARN_CFG)

rec_t1 = run_train_split(TRAIN_TASK1_DIR, SLOVED_TASK1, learner, hints_csv=HINTS_TRAIN1)
rec_t2 = run_train_split(TRAIN_TASK2_DIR, SLOVED_TASK2, learner, hints_csv=None)

all_bands    = [b for (b, pw) in (rec_t1 + rec_t2)]
all_prior_ws = [pw for (b, pw) in (rec_t1 + rec_t2)]
global_band    = int(mode_select(all_bands) if all_bands else 6)
global_prior_w = float(mode_select(all_prior_ws) if all_prior_ws else learner.get_w()["prior_w"])

TRAINED_MODEL = {
    "weights": learner.get_w(),      # alpha..lbp, prior_w (EMA)
    "band": int(global_band),        # band chọn theo mode
    "prior_w": float(global_prior_w) # prior_w chọn theo mode (ổn định)
}
with open(MODEL_PATH, "w", encoding="utf-8") as f:
    json.dump(TRAINED_MODEL, f, ensure_ascii=False, indent=2)
logger.info(f"[TRAIN DONE] Saved trained model to {MODEL_PATH}: {TRAINED_MODEL}")
print("✅ Cell 11 ready.")


2025-10-29 04:54:56,076 | INFO | ===== TRAINING PHASE START =====
2025-10-29 04:54:56,078 | WARNING | Hint file not found (optional): /home/dammanhdungvn/learn_ml/project/output/task1_hints.csv


TRAIN on task1:   0%|                                                                                          | 0/8 [00:00<?, ?it/s]/home/dammanhdungvn/learn_ml/venv/lib/python3.12/site-packages/skimage/feature/texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


2025-10-29 04:55:00,390 | INFO | origin1.png | expansions=5000, frontier=10022
2025-10-29 04:55:00,773 | INFO | origin1.png | expansions=10000, frontier=13827
2025-10-29 04:55:01,064 | INFO | origin1.png | expansions=15000, frontier=15978
2025-10-29 04:55:01,306 | INFO | origin1.png | expansions=20000, frontier=14681
2025-10-29 04:55:01,500 | INFO | origin1.png | expansions=25000, frontier=10659
2025-10-29 04:55:01,593 | INFO | origin1.png | A* done | expansions=28526 | best_cost=53.122688
2025-10-29 04:55:05,973 | INFO | origin1.png | expansions=5000, frontier=10128
2025-10-29 04:55:06,301 | INFO | origin1.png | expansions=10000, frontier=13989
2025-10-29 04:55:06,595 | INFO | origin1.png | expansions=15000, frontier=16036
2025-10-29 04:55:06,822 | INFO | origin1.png | expansions=20000, frontier=14808
2025-10-29 04:55:06,993 | INFO | origin1.png | expansions=25000, frontier=10723
2025-10-29 04:55:07,083 | INFO | origin1.png | A* done | expansions=28156 | best_cost=52.615440
2025-10-29

TRAIN on task1:  12%|██████████▎                                                                       | 1/8 [01:11<08:22, 71.73s/it]

2025-10-29 04:56:12,266 | INFO | origin2.png | expansions=5000, frontier=10548
2025-10-29 04:56:12,604 | INFO | origin2.png | expansions=10000, frontier=14264
2025-10-29 04:56:12,883 | INFO | origin2.png | expansions=15000, frontier=15817
2025-10-29 04:56:13,097 | INFO | origin2.png | expansions=20000, frontier=14336
2025-10-29 04:56:13,287 | INFO | origin2.png | expansions=25000, frontier=10581
2025-10-29 04:56:13,317 | INFO | origin2.png | A* done | expansions=25661 | best_cost=48.474266
2025-10-29 04:56:17,516 | INFO | origin2.png | expansions=5000, frontier=10565
2025-10-29 04:56:17,902 | INFO | origin2.png | expansions=10000, frontier=14333
2025-10-29 04:56:18,212 | INFO | origin2.png | expansions=15000, frontier=15887
2025-10-29 04:56:18,455 | INFO | origin2.png | expansions=20000, frontier=14270
2025-10-29 04:56:18,632 | INFO | origin2.png | expansions=25000, frontier=10626
2025-10-29 04:56:18,647 | INFO | origin2.png | A* done | expansions=25430 | best_cost=47.998035
2025-10-29

...

2025-10-29 05:46:46,037 | INFO | origin7.png | expansions=5000, frontier=10320
2025-10-29 05:46:46,370 | INFO | origin7.png | expansions=10000, frontier=13776
2025-10-29 05:46:46,645 | INFO | origin7.png | expansions=15000, frontier=17583
2025-10-29 05:46:46,928 | INFO | origin7.png | expansions=20000, frontier=16780
2025-10-29 05:46:47,094 | INFO | origin7.png | expansions=25000, frontier=12218
2025-10-29 05:46:47,220 | INFO | origin7.png | expansions=30000, frontier=5144
2025-10-29 05:46:47,271 | INFO | origin7.png | A* done | expansions=32752 | best_cost=71.184227
2025-10-29 05:46:51,280 | INFO | origin7.png | expansions=5000, frontier=10356
2025-10-29 05:46:51,610 | INFO | origin7.png | expansions=10000, frontier=13813
2025-10-29 05:46:51,927 | INFO | origin7.png | expansions=15000, frontier=17624
2025-10-29 05:46:52,157 | INFO | origin7.png | expansions=20000, frontier=16804
2025-10-29 05:46:52,332 | INFO | origin7.png | expansions=25000, frontier=12206
2025-10-29 05:46:52,469 | I

TRAIN on task2:  88%|██████████████████████████████████████████████████████████████████████▉          | 7/8 [43:10<06:46, 406.70s/it]

2025-10-29 05:47:58,211 | INFO | origin8.png | expansions=5000, frontier=11879
2025-10-29 05:47:58,538 | INFO | origin8.png | expansions=10000, frontier=16024
2025-10-29 05:47:58,797 | INFO | origin8.png | expansions=15000, frontier=17769
2025-10-29 05:47:59,039 | INFO | origin8.png | expansions=20000, frontier=16071
2025-10-29 05:47:59,277 | INFO | origin8.png | expansions=25000, frontier=12029
2025-10-29 05:47:59,317 | INFO | origin8.png | A* done | expansions=26187 | best_cost=41.793369
2025-10-29 05:48:03,843 | INFO | origin8.png | expansions=5000, frontier=12005
2025-10-29 05:48:04,235 | INFO | origin8.png | expansions=10000, frontier=16122
2025-10-29 05:48:04,554 | INFO | origin8.png | expansions=15000, frontier=17978
2025-10-29 05:48:04,798 | INFO | origin8.png | expansions=20000, frontier=16130
2025-10-29 05:48:04,967 | INFO | origin8.png | expansions=25000, frontier=12100
2025-10-29 05:48:04,996 | INFO | origin8.png | A* done | expansions=25994 | best_cost=41.405010
2025-10-29

TRAIN on task2: 100%|█████████████████████████████████████████████████████████████████████████████████| 8/8 [44:23<00:00, 332.89s/it]

2025-10-29 05:49:06,184 | INFO | [TRAIN DONE] Saved trained model to /home/dammanhdungvn/learn_ml/project/output/trained_model.json: {'weights': {'alpha': 0.9794236618295111, 'beta': 0.34002558500460633, 'gamma': 0.8000000110357872, 'ori': 0.10807164054125219, 'lbp': 0.10092364767692683, 'prior_w': 0.07448736027954495}, 'band': 5, 'prior_w': 0.1}
✅ Cell 11 ready.


In [27]:
# ============================================================
# Cell 12 — INFERENCE: PUBLIC TEST (NO FINE-TUNE, NO LEARNING)
# ------------------------------------------------------------
# - Đọc model JSON (đã freeze) -> w, band, prior_w cố định.
# - Giải các PNG trong images/test (không tách task).
# - Hỗ trợ hint CSV (nếu có), mặc định None.
# - Xuất output/public_output.csv theo schema chấm.
# ============================================================

def solve_dir_fixed_params(test_dir, out_dir, out_csv, hints_csv=None, model_json=MODEL_PATH):
    try:
        require_dir(test_dir, create=False)
    except Exception:
        logger.error(f"[INFER] Test dir not found, skip: {test_dir}")
        return pd.DataFrame([])

    os.makedirs(out_dir, exist_ok=True)
    files = list_images(test_dir)
    if not files:
        logger.warning(f"[INFER] No PNG found in: {test_dir}")

    # Load model (đã freeze)
    with open(model_json, "r", encoding="utf-8") as f:
        mdl = json.load(f)
    w       = mdl["weights"]
    band    = int(mdl["band"])
    prior_w = float(mdl["prior_w"])
    hints = load_hints_csv(hints_csv) if hints_csv else {}

    rows = []
    for path in tqdm(files, desc=f"INFER on {os.path.basename(test_dir)}"):
        fname = os.path.basename(path)
        try:
            initial_piece = hints.get(fname, None)
            perm, cost, vis, tiles, feats, costs, prior = solve_one_image_with_params(
                path, w, band, prior_w, initial_piece=initial_piece, do_local_improve=True
            )
            cv2.imwrite(os.path.join(out_dir, fname), cv2.cvtColor(vis, cv2.COLOR_RGB2BGR))
            rows.append([fname] + perm)
            logger.info(f"[INFER:{fname}] cost={cost:.6f}")
        except Exception:
            logger.exception(f"[INFER:{fname}] error")

    cols = ["image_filename"] + [f"piece_at_{r}_{c}" for r in range(GRID_R) for c in range(GRID_C)]
    df = pd.DataFrame(rows, columns=cols)
    df.sort_values("image_filename", inplace=True)
    df.to_csv(out_csv, index=False, encoding="utf-8")
    logger.info(f"[INFER] Wrote {out_csv} with {len(df)} rows.")
    return df

logger.info("===== INFERENCE: PUBLIC TEST =====")
df_public = solve_dir_fixed_params(
    TEST_PUBLIC_DIR, SLOVED_PUBLIC, PUBLIC_CSV,
    hints_csv=HINTS_PUBLIC, model_json=MODEL_PATH
)
print("✅ Cell 12 ready.")


2025-10-29 05:55:14,260 | INFO | ===== INFERENCE: PUBLIC TEST =====
2025-10-29 05:55:14,262 | ERROR | [INFER] Test dir not found, skip: /home/dammanhdungvn/learn_ml/project/images/test
✅ Cell 12 ready.


In [26]:
# ============================================================
# Cell 13 — INFERENCE: PRIVATE TEST (NO FINE-TUNE, NO LEARNING)
# ------------------------------------------------------------
# - Nếu thư mục private_test chưa mở khoá -> log và bỏ qua.
# - Nếu có HINTS_PRIVATE thì đọc, không thì None.
# - Xuất output/private_output.csv theo schema.
# ============================================================

logger.info("===== INFERENCE: PRIVATE TEST =====")
try:
    require_dir(PRIVATE_TEST_DIR, create=False)  # có thể chưa mở khoá
    df_private = solve_dir_fixed_params(
        PRIVATE_TEST_DIR, SLOVED_PRIVATE, PRIVATE_CSV,
        hints_csv=HINTS_PRIVATE, model_json=MODEL_PATH
    )
except Exception:
    logger.warning(f"[INFER] Private test dir missing or locked. Skip. Path={PRIVATE_TEST_DIR}")
print("✅ Cell 13 ready.")


2025-10-29 05:55:08,448 | INFO | ===== INFERENCE: PRIVATE TEST =====
2025-10-29 05:55:08,450 | WARNING | Hint file not found (optional): /home/dammanhdungvn/learn_ml/project/output/private_task1_hints.csv


INFER on private_test:   0%|                                                                                   | 0/1 [00:00<?, ?it/s]

2025-10-29 05:55:12,755 | INFO | origin6.png | expansions=5000, frontier=10406
2025-10-29 05:55:13,118 | INFO | origin6.png | expansions=10000, frontier=14476
2025-10-29 05:55:13,469 | INFO | origin6.png | expansions=15000, frontier=16181
2025-10-29 05:55:13,725 | INFO | origin6.png | expansions=20000, frontier=15165
2025-10-29 05:55:13,918 | INFO | origin6.png | expansions=25000, frontier=11412
2025-10-29 05:55:14,040 | INFO | origin6.png | A* done | expansions=29292 | best_cost=55.973598
2025-10-29 05:55:14,089 | INFO | [INFER:origin6.png] cost=55.973595


INFER on private_test: 100%|███████████████████████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.64s/it]

2025-10-29 05:55:14,094 | INFO | [INFER] Wrote /home/dammanhdungvn/learn_ml/project/output/private_output.csv with 1 rows.
✅ Cell 13 ready.


In [20]:
# ============================================================
# Cell 14 — PACKAGE SUBMISSION
# ------------------------------------------------------------
# - Đóng gói public_output.csv + (private_output.csv nếu có)
#   + Notebook (.ipynb) + trained_model.json vào submission.zip.
# ============================================================

NOTEBOOK_FILENAME = "solver_voai_task2_train_freeze_infer.ipynb"  # ← sửa đúng tên file notebook của bạn

with zipfile.ZipFile(SUBMISSION, 'w', compression=zipfile.ZIP_DEFLATED) as zf:
    for p in [PUBLIC_CSV, PRIVATE_CSV, MODEL_PATH]:
        if os.path.isfile(p):
            zf.write(p, arcname=os.path.basename(p))
    nb_local = os.path.join(os.getcwd(), NOTEBOOK_FILENAME)
    nb_alt   = os.path.join(os.path.join(BASE_DIR, "code"), NOTEBOOK_FILENAME)
    nb_path  = nb_local if os.path.isfile(nb_local) else nb_alt
    if os.path.isfile(nb_path):
        zf.write(nb_path, arcname=NOTEBOOK_FILENAME)

logger.info(f"Created {SUBMISSION}.")
print("✅ DONE: Training → Freeze → Public/Private inference completed.")


2025-10-29 05:52:58,699 | INFO | Created /home/dammanhdungvn/learn_ml/project/output/submission.zip.
✅ DONE: Training → Freeze → Public/Private inference completed.
